In [1]:
from ClusterPrepForDFT import *

In [2]:
from ClusterPrepForDFT.capping import *

## Wustite

In [5]:
# coordination number of each atom
numnearest = {
    'Fe': 6,
    'O': 6
}

# distance to put the capping hydrogen from the surface atom
cappingdistances = {
    'Fe': 2.03,
    'O': 0.98
}

#charge of Bq charge
#note, this is the atom being replaced, the atom interior, and then the bq charge
bqchargemap = {
    'O':{'Fe': +2/6 - 1}, # +2 charge / 6 bonds - 1 to override capping H
    'Fe':{'O': -2/6 - 1} # -2 charge / 6 bonds - 1 to override capping H
}

In [6]:
#this makes the files.  The inputs are (the basename of the .xyz/.bq files, the .cif file's name ..., the central atom, and the radius of cluster) 
capped = make_hydrogen_capped_cluster_from_cif_2('Wustite_capped',
                                               'wustite.cif', 
                                                numnearest, 
                                                cappingdistances, 
                                                bqchargemap, 
                                                'Fe',
                                                sphereradius=4)

Importing structure
Finding site inside/outside sphere.
Writing capped cluster file:  Wustite_capped.xyz
Writing bq charge file:  Wustite_capped.bq


## Hematite

In [7]:
# coordination number of each atom
numnearest = {
    'Fe': 6,
    'O': 4
}

# distance to put the capping hydrogen from the surface atom
cappingdistances = {
    'Fe': 2.03,
    'O': 0.98
}

#charge of Bq charge
#note, this is the atom being replaced, the atom interior, and then the bq charge
bqchargemap = {
    'O':{'Fe': +3/6 - 1}, # +2 charge / 6 bonds - 1 to override capping H
    'Fe':{'O': -2/4 - 1} # -2 charge / 6 bonds - 1 to override capping H
}

In [8]:
#this makes the files.  The inputs are (the basename of the .xyz/.bq files, the .cif file's name ..., the central atom, and the radius of cluster) 
capped = make_hydrogen_capped_cluster_from_cif_2('Hematite_capped',
                                               'hematite.cif', 
                                                numnearest, 
                                                cappingdistances, 
                                                bqchargemap, 
                                                'Fe',
                                                sphereradius=5)

Importing structure
Finding site inside/outside sphere.
Writing capped cluster file:  Hematite_capped.xyz
Writing bq charge file:  Hematite_capped.bq


## Aegirine

In [9]:
outfilename='Aegirine_capped'
ciffile = 'aegirine.cif'
bondlength= {'r':1.8,'dr':0.4}
cappingdistances= {
    'Fe': 2.03,
    'O': 0.98,
    'Si': 1.49
}
bqchargemap= {
    'O':{'Fe':'checkdist','Si':'checkdist'},
    'Fe':{'O':'checkdist'},
    'Si':{'O':'checkdist'},#note, this is the atom being replaced, the atom interior, and then the bq charge
}

radialbins = [1.4, 1.6, 1.63, 1.65, 2.0, 2.2]

bq4unique= {
    'O':{1:+4/4-1,  2:+4/4-1, 3:+4/4-1, 4:+3/6-1, 5:+3/6-1}, #note, this is the atom being replaced and whether the oxygen involved is one, two, or three-fold coordinated
    'Fe':{1:0,      2:0,      3:0,      4:-3/6-1, 5:-3/6-1},
    'Si':{1:-4/4-1, 2:-4/4-1, 3:-4/4-1, 4:0, 5:0}
}

centeratom='Fe' 
supercell=5


In [10]:
capped = make_hydrogen_capped_cluster_from_cif_3(outfilename,
                                            ciffile,
                                            radialbins,
                                            bondlength, 
                                            cappingdistances, 
                                            bqchargemap,
                                            bq4unique, 
                                            'Fe',
                                            sphereradius=5.5,
                                            supercell=6)

Importing structure
Finding site inside/outside sphere.
Writing capped cluster file:  Aegirine_capped.xyz
Writing bq charge file:  Aegirine_capped.bq


## Olivine

### Sym 1

In [13]:
def tag_sites_by_equivalency(struct):
    sga = mg.symmetry.analyzer.SpacegroupAnalyzer(struct)
    print('getting symmetrized structure')
    ss = sga.get_symmetrized_structure()
    
    i = 0
    for eset in ss.equivalent_sites:
        print('doing site {}'.format(i), end='\r')
        for s in eset:
            s.tag = i
        i += 1
        
    mol = mg.Molecule.from_sites(ss.sites)
    for i, site in enumerate(ss):
        print('tagging site {} out of {}'.format(i, len(ss)), end='\r')
        new = mol.get_sites_in_sphere(site.coords, 1e-10)[0][0]
        new.tag = site.tag
        mol[i].tag=new.tag
        
    return mol

In [14]:
outfilename='Olivine_sym1_capped'
ciffile = 'olivine.cif'
bondlength= {'r':1.8,'dr':0.4}
cappingdistances= {
    'Fe': 2.03,
    'O': 0.98,
    'Si': 1.49
}
bqchargemap= {
    'O':{'Fe':+2/6-1,'Si':+4/4-1},
    'Fe':{'O':-2/6-1},
    'Si':{'O':-4/4-1},#note, this is the atom being replaced, the atom interior, and then the bq charge
}
bq4unique= {
    'O':{1:0}, #note, this is the atom being replaced and whether the oxygen involved is one, two, or three-fold coordinated
    'Fe':{1:0},
}
centeratom='Fe' 
supercell=5
radialbins = [1.4, 2.2]

In [15]:
'''Make hydrogen-capped cluster.

Supercell needs to be big enough that all atoms in sphere and neighbors thereof
have full neighbors.'''
print('Importing structure')
struct = mg.Structure.from_file(ciffile)
struct.make_supercell(supercell)

Importing structure


In [16]:
newmol = tag_sites_by_equivalency(struct)

getting symmetrized structure


In [17]:
for site in newmol:
    if site.tag == 0:
        print(site.specie, site.coords, site.tag)

Fe [0. 0. 0.] 0
Fe [0.    0.    6.086] 0
Fe [ 0.     0.    12.172] 0
Fe [ 0.     0.    18.258] 0
Fe [ 0.     0.    24.344] 0
Fe [1.68370441e-15 1.04700000e+01 6.41102599e-16] 0
Fe [1.68370441e-15 1.04700000e+01 6.08600000e+00] 0
Fe [1.68370441e-15 1.04700000e+01 1.21720000e+01] 0
Fe [1.68370441e-15 1.04700000e+01 1.82580000e+01] 0
Fe [1.68370441e-15 1.04700000e+01 2.43440000e+01] 0
Fe [3.36740883e-15 2.09400000e+01 1.28220520e-15] 0
Fe [3.36740883e-15 2.09400000e+01 6.08600000e+00] 0
Fe [3.36740883e-15 2.09400000e+01 1.21720000e+01] 0
Fe [3.36740883e-15 2.09400000e+01 1.82580000e+01] 0
Fe [3.36740883e-15 2.09400000e+01 2.43440000e+01] 0
Fe [5.05111324e-15 3.14100000e+01 3.04300000e+01] 0
Fe [5.05111324e-15 3.14100000e+01 6.08600000e+00] 0
Fe [5.05111324e-15 3.14100000e+01 1.21720000e+01] 0
Fe [5.05111324e-15 3.14100000e+01 1.82580000e+01] 0
Fe [5.05111324e-15 3.14100000e+01 2.43440000e+01] 0
Fe [6.73481766e-15 4.18800000e+01 2.56441040e-15] 0
Fe [6.73481766e-15 4.18800000e+01 6.0860000

Fe [12.045  5.235 30.43 ] 0
Fe [12.045  5.235  6.086] 0
Fe [12.045  5.235 12.172] 0
Fe [12.045  5.235 18.258] 0
Fe [12.045  5.235 24.344] 0
Fe [1.20450000e+01 1.57050000e+01 1.69919743e-15] 0
Fe [12.045 15.705  6.086] 0
Fe [12.045 15.705 12.172] 0
Fe [12.045 15.705 18.258] 0
Fe [12.045 15.705 24.344] 0
Fe [12.045 26.175 30.43 ] 0
Fe [12.045 26.175  6.086] 0
Fe [12.045 26.175 12.172] 0
Fe [12.045 26.175 18.258] 0
Fe [12.045 26.175 24.344] 0
Fe [12.045 36.645 30.43 ] 0
Fe [12.045 36.645  6.086] 0
Fe [12.045 36.645 12.172] 0
Fe [12.045 36.645 18.258] 0
Fe [12.045 36.645 24.344] 0
Fe [1.20450000e+01 4.71150000e+01 3.62250523e-15] 0
Fe [12.045 47.115  6.086] 0
Fe [12.045 47.115 12.172] 0
Fe [12.045 47.115 18.258] 0
Fe [12.045 47.115 24.344] 0
Fe [1.68630000e+01 5.23500000e+00 1.35311225e-15] 0
Fe [16.863  5.235  6.086] 0
Fe [16.863  5.235 12.172] 0
Fe [16.863  5.235 18.258] 0
Fe [16.863  5.235 24.344] 0
Fe [1.68630000e+01 1.57050000e+01 1.99421485e-15] 0
Fe [16.863 15.705  6.086] 0
Fe [16.8

In [18]:
sym=0
r = 0.1
while True:
    print(r)
    sites = newmol.get_sites_in_sphere(newmol.center_of_mass, r)
    found=[]
    indices=[]
    try:
        for s in sites:
            i=0
            for site in newmol:
                if all(site.coords==s.coords):
                    found.append(newmol[i].tag == sym)
                    indices.append(i)
                    break
                i+=1
    except AttributeError:
        for s in sites:
            i=0
            for site in newmol:
                if all(site.coords==s.coords):
                    found.append(newmol[i].tag == sym)
                    break
                i+=1
    if any(found):
        break
    r += 0.1
centerindex=indices[np.where(found)[0][0]]

0.1
0.2
0.30000000000000004
0.4
0.5
0.6
0.7
0.7999999999999999


In [19]:
centerindex

312

In [20]:
center = newmol[centerindex]

In [21]:
mol = mg.Molecule(struct.species, struct.cart_coords)

In [22]:
mol.translate_sites(vector=-center.coords)

In [23]:
sphereradius=5.3

In [24]:
remove_oxidation_state_from_species(mol)

#answer = input('Is this a site: {}'.format(sitedata))

print('Finding site inside/outside sphere.')
innersites = mol.get_sites_in_sphere([0, 0, 0], sphereradius)
inner = mg.Molecule.from_sites([s[0] for s in innersites])

neighboringmol = mg.Molecule([], [])
for isite in inner:
    print('Checking neighbors of site {}'.format(mol.index(isite)), end='\r')
    neighbors = mol.get_neighbors_in_shell(isite.coords, bondlength['r'],bondlength['dr'])
    for s in neighbors:
        if s not in inner:  

            if s not in neighboringmol:
                neighboringmol.append(s.specie, s.coords)

            try:
                neighboringmol[neighboringmol.index(s)].innerneighbors.append(isite)
            except AttributeError:
                neighboringmol[neighboringmol.index(s)].innerneighbors = [isite] # I don't understand what this except does

hydrcoords = []
hydrneighbors = []
hydrreplace = []
for ns in neighboringmol:
    if ns.specie.name == 'Li':
        continue
    for n in ns.innerneighbors:
        if n.specie.name == 'Li':
            continue
        hydrcoords.append(np.mean([ns.coords, n.coords], axis=0)) # create hydrogen halfway along bond
        hydrneighbors.append(n) # keep track of neighbor identity
        hydrreplace.append(ns) # keep track of identitiy of atom being replaced

hydrsites = [mg.Site('H', c) for c in hydrcoords]
hydr = mg.Molecule.from_sites(hydrsites)

# add hydrogen neighbors and adjust bond lengths accordingly
for hs, hn, hr in zip(hydrsites, hydrneighbors, hydrreplace):
    hydr[hydr.index(hs)].innerneighbor = hn
    hydr[hydr.index(hs)].replacing = hr
for s in hydr:
    adjust_bond_length(s, s.innerneighbor, cappingdistances[str(s.innerneighbor.specie)])

capped = mg.Molecule.from_sites(hydr.sites + inner.sites)
print('Writing capped cluster file: ', '{}.xyz'.format(outfilename))
capped.to('xyz', '{}.xyz'.format(outfilename))

print('Writing bq charge file: ', '{}.bq'.format(outfilename))
totalbqcharge = 0
with open('{}.bq'.format(outfilename), 'w') as file:
    for s in hydr:
        #print(str(s.replacing.specie), str(s.innerneighbor.specie))
        bq = bqchargemap[str(s.replacing.specie)][str(s.innerneighbor.specie)]
        if bq == 'checkdist':
            dist = np.linalg.norm(s.replacing.coords - s.innerneighbor.coords)
            index = np.digitize(dist, radialbins)
            bq = bq4unique[str(s.replacing.specie)][int(index)]
        file.write('Bq {:>15.6f}{:>15.6f}{:>15.6f}{:>10.4f}\n'.format(
            *s.coords, bq))
        totalbqcharge += bq

with open('{}.xyz'.format(outfilename), 'r') as file:
    oldlines = file.readlines()

oldlines[1] = oldlines[1].strip() + '; Total bqcharge for capped cluster: {}\n'.format(totalbqcharge)
with open('{}.xyz'.format(outfilename), 'w') as file:
    for line in oldlines:
        file.write(line)

slightly_less_dumb_rename_center_atom('{}.xyz'.format(outfilename))

Finding site inside/outside sphere.
Writing capped cluster file:  Olivine_sym1_capped.xyz
Writing bq charge file:  Olivine_sym1_capped.bq


In [25]:
totalbqcharge

-64.00000000000003

### Sym 2

In [26]:
outfilename='Olivine_sym2_capped'
ciffile = 'olivine.cif'
bondlength= {'r':1.8,'dr':0.4}
cappingdistances= {
    'Fe': 2.03,
    'O': 0.98,
    'Si': 1.49
}
bqchargemap= {
    'O':{'Fe':+2/6-1,'Si':+4/4-1},
    'Fe':{'O':-2/6-1},
    'Si':{'O':-4/4-1},#note, this is the atom being replaced, the atom interior, and then the bq charge
}
bq4unique= {
    'O':{1:0}, #note, this is the atom being replaced and whether the oxygen involved is one, two, or three-fold coordinated
    'Fe':{1:0},
}
centeratom='Fe' 
supercell=5
radialbins = [1.4, 2.2]

In [27]:
'''Make hydrogen-capped cluster.

Supercell needs to be big enough that all atoms in sphere and neighbors thereof
have full neighbors.'''
print('Importing structure')
struct = mg.Structure.from_file(ciffile)
struct.make_supercell(supercell)

Importing structure


In [28]:
newmol = tag_sites_by_equivalency(struct)

getting symmetrized structure


In [29]:
for site in newmol:
    if site.tag == 1:
        print(site.specie, site.coords, site.tag)

Fe [4.74573 2.92113 1.5215 ] 1
Fe [4.74573 2.92113 7.6075 ] 1
Fe [ 4.74573  2.92113 13.6935 ] 1
Fe [ 4.74573  2.92113 19.7795 ] 1
Fe [ 4.74573  2.92113 25.8655 ] 1
Fe [ 4.74573 13.39113  1.5215 ] 1
Fe [ 4.74573 13.39113  7.6075 ] 1
Fe [ 4.74573 13.39113 13.6935 ] 1
Fe [ 4.74573 13.39113 19.7795 ] 1
Fe [ 4.74573 13.39113 25.8655 ] 1
Fe [ 4.74573 23.86113  1.5215 ] 1
Fe [ 4.74573 23.86113  7.6075 ] 1
Fe [ 4.74573 23.86113 13.6935 ] 1
Fe [ 4.74573 23.86113 19.7795 ] 1
Fe [ 4.74573 23.86113 25.8655 ] 1
Fe [ 4.74573 34.33113  1.5215 ] 1
Fe [ 4.74573 34.33113  7.6075 ] 1
Fe [ 4.74573 34.33113 13.6935 ] 1
Fe [ 4.74573 34.33113 19.7795 ] 1
Fe [ 4.74573 34.33113 25.8655 ] 1
Fe [ 4.74573 44.80113  1.5215 ] 1
Fe [ 4.74573 44.80113  7.6075 ] 1
Fe [ 4.74573 44.80113 13.6935 ] 1
Fe [ 4.74573 44.80113 19.7795 ] 1
Fe [ 4.74573 44.80113 25.8655 ] 1
Fe [9.56373 2.92113 1.5215 ] 1
Fe [9.56373 2.92113 7.6075 ] 1
Fe [ 9.56373  2.92113 13.6935 ] 1
Fe [ 9.56373  2.92113 19.7795 ] 1
Fe [ 9.56373  2.92113 25.8

Fe [16.79073 23.25387  4.5645 ] 1
Fe [16.79073 23.25387 10.6505 ] 1
Fe [16.79073 23.25387 16.7365 ] 1
Fe [16.79073 23.25387 22.8225 ] 1
Fe [16.79073 23.25387 28.9085 ] 1
Fe [16.79073 33.72387  4.5645 ] 1
Fe [16.79073 33.72387 10.6505 ] 1
Fe [16.79073 33.72387 16.7365 ] 1
Fe [16.79073 33.72387 22.8225 ] 1
Fe [16.79073 33.72387 28.9085 ] 1
Fe [16.79073 44.19387  4.5645 ] 1
Fe [16.79073 44.19387 10.6505 ] 1
Fe [16.79073 44.19387 16.7365 ] 1
Fe [16.79073 44.19387 22.8225 ] 1
Fe [16.79073 44.19387 28.9085 ] 1
Fe [21.60873  2.31387  4.5645 ] 1
Fe [21.60873  2.31387 10.6505 ] 1
Fe [21.60873  2.31387 16.7365 ] 1
Fe [21.60873  2.31387 22.8225 ] 1
Fe [21.60873  2.31387 28.9085 ] 1
Fe [21.60873 12.78387  4.5645 ] 1
Fe [21.60873 12.78387 10.6505 ] 1
Fe [21.60873 12.78387 16.7365 ] 1
Fe [21.60873 12.78387 22.8225 ] 1
Fe [21.60873 12.78387 28.9085 ] 1
Fe [21.60873 23.25387  4.5645 ] 1
Fe [21.60873 23.25387 10.6505 ] 1
Fe [21.60873 23.25387 16.7365 ] 1
Fe [21.60873 23.25387 22.8225 ] 1
Fe [21.60873 2

In [30]:
sym=1
r = 0.1
while True:
    print(r)
    sites = newmol.get_sites_in_sphere(newmol.center_of_mass, r)
    found=[]
    indices=[]
    try:
        for s in sites:
            i=0
            for site in newmol:
                if all(site.coords==s.coords):
                    found.append(newmol[i].tag == sym)
                    indices.append(i)
                    break
                i+=1
    except AttributeError:
        for s in sites:
            i=0
            for site in newmol:
                if all(site.coords==s.coords):
                    found.append(newmol[i].tag == sym)
                    break
                i+=1
    if any(found):
        break
    r += 0.1
centerindex=indices[np.where(found)[0][0]]

0.1
0.2
0.30000000000000004
0.4
0.5
0.6
0.7
0.7999999999999999
0.8999999999999999
0.9999999999999999
1.0999999999999999
1.2
1.3
1.4000000000000001
1.5000000000000002
1.6000000000000003
1.7000000000000004
1.8000000000000005
1.9000000000000006
2.0000000000000004
2.1000000000000005
2.2000000000000006
2.3000000000000007
2.400000000000001
2.500000000000001
2.600000000000001
2.700000000000001
2.800000000000001


In [31]:
centerindex

812

In [32]:
center = newmol[centerindex]

In [33]:
mol = mg.Molecule(struct.species, struct.cart_coords)

In [34]:
mol.translate_sites(vector=-center.coords)

In [35]:
sphereradius=5.3

In [36]:
remove_oxidation_state_from_species(mol)

#answer = input('Is this a site: {}'.format(sitedata))

print('Finding site inside/outside sphere.')
innersites = mol.get_sites_in_sphere([0, 0, 0], sphereradius)
inner = mg.Molecule.from_sites([s[0] for s in innersites])

neighboringmol = mg.Molecule([], [])
for isite in inner:
    print('Checking neighbors of site {}'.format(mol.index(isite)), end='\r')
    neighbors = mol.get_neighbors_in_shell(isite.coords, bondlength['r'],bondlength['dr'])
    for s in neighbors:
        if s not in inner:  

            if s not in neighboringmol:
                neighboringmol.append(s.specie, s.coords)

            try:
                neighboringmol[neighboringmol.index(s)].innerneighbors.append(isite)
            except AttributeError:
                neighboringmol[neighboringmol.index(s)].innerneighbors = [isite] # I don't understand what this except does

hydrcoords = []
hydrneighbors = []
hydrreplace = []
for ns in neighboringmol:
    if ns.specie.name == 'Li':
        continue
    for n in ns.innerneighbors:
        if n.specie.name == 'Li':
            continue
        hydrcoords.append(np.mean([ns.coords, n.coords], axis=0)) # create hydrogen halfway along bond
        hydrneighbors.append(n) # keep track of neighbor identity
        hydrreplace.append(ns) # keep track of identitiy of atom being replaced

hydrsites = [mg.Site('H', c) for c in hydrcoords]
hydr = mg.Molecule.from_sites(hydrsites)

# add hydrogen neighbors and adjust bond lengths accordingly
for hs, hn, hr in zip(hydrsites, hydrneighbors, hydrreplace):
    hydr[hydr.index(hs)].innerneighbor = hn
    hydr[hydr.index(hs)].replacing = hr
for s in hydr:
    adjust_bond_length(s, s.innerneighbor, cappingdistances[str(s.innerneighbor.specie)])

capped = mg.Molecule.from_sites(hydr.sites + inner.sites)
print('Writing capped cluster file: ', '{}.xyz'.format(outfilename))
capped.to('xyz', '{}.xyz'.format(outfilename))

print('Writing bq charge file: ', '{}.bq'.format(outfilename))
totalbqcharge = 0
with open('{}.bq'.format(outfilename), 'w') as file:
    for s in hydr:
        #print(str(s.replacing.specie), str(s.innerneighbor.specie))
        bq = bqchargemap[str(s.replacing.specie)][str(s.innerneighbor.specie)]
        if bq == 'checkdist':
            dist = np.linalg.norm(s.replacing.coords - s.innerneighbor.coords)
            index = np.digitize(dist, radialbins)
            bq = bq4unique[str(s.replacing.specie)][int(index)]
        file.write('Bq {:>15.6f}{:>15.6f}{:>15.6f}{:>10.4f}\n'.format(
            *s.coords, bq))
        totalbqcharge += bq

with open('{}.xyz'.format(outfilename), 'r') as file:
    oldlines = file.readlines()

oldlines[1] = oldlines[1].strip() + '; Total bqcharge for capped cluster: {}\n'.format(totalbqcharge)
with open('{}.xyz'.format(outfilename), 'w') as file:
    for line in oldlines:
        file.write(line)

slightly_less_dumb_rename_center_atom('{}.xyz'.format(outfilename))

Finding site inside/outside sphere.
Writing capped cluster file:  Olivine_sym2_capped.xyz
Writing bq charge file:  Olivine_sym2_capped.bq


In [37]:
totalbqcharge

-80.0

## Staurolite

In [38]:
# coordination number of each atom
numnearest = {
    'Fe': 4,
    'O': 4,
    'Si': 4,
    'Al': 6
}

# distance to put the capping hydrogen from the surface atom
cappingdistances = {
    'Fe': 2.03,
    'O': 0.98,
    'Si': 1.49,
    'Al': 1.93
}

#charge of Bq charge
#note, this is the atom being replaced, the atom interior, and then the bq charge
bqchargemap = {
    'O':{'Fe': +2/4 - 1,'Si':+4/4 - 1 ,'Al':+3/6 -1}, # +2 charge / 6 bonds - 1 to override capping H
    'Fe':{'O': -2/4 - 1}, # -2 charge / 6 bonds - 1 to override capping H
    'Si':{'O':-4/4 - 1},
    'Al':{'O':-3/6 - 1}
}

In [41]:
#this makes the files.  The inputs are (the basename of the .xyz/.bq files, the .cif file's name ..., the central atom, and the radius of cluster) 
capped = make_hydrogen_capped_cluster_from_cif_2('Staurolite_capped',
                                               'staurolite.cif', 
                                                numnearest, 
                                                cappingdistances, 
                                                bqchargemap, 
                                                'Fe',
                                                supercell=5,
                                                sphereradius=2.5)

Importing structure
Finding site inside/outside sphere.
Writing capped cluster file:  Staurolite_capped.xyz
Writing bq charge file:  Staurolite_capped.bq


### Capping length-based style

In [42]:
cappingdistances = {
    'Fe': 2.03,
    'O': 0.98,
    'Si': 1.49,
    'Al': 1.93
}

#checkdist tells the program that the bond involves atoms which do not have unique symmetry sties and that it should defer to bq4unique to determine the bqcharge to place.
bqchargemap = {
    'O':{'Fe': +2/4 - 1,'Si':+4/4 - 1 ,'Al':+3/6 -1}, # +2 charge / 6 bonds - 1 to override capping H
    'Fe':{'O': -2/4 - 1}, # -2 charge / 6 bonds - 1 to override capping H
    'Si':{'O':-4/4 - 1},
    'Al':{'O':-3/6 - 1}
}

bq4unique = {
}

#looks for atoms within r+/-dr of the interior atom to decide they are bonding 
bondlength = {'r':1.8,'dr':0.4}

In [43]:
capped = make_hydrogen_capped_cluster_from_cif_3('Staurolite3_capped',
                                               'staurolite.cif',
                                                [0,3],
                                                bondlength, 
                                                cappingdistances, 
                                                bqchargemap,
                                                bq4unique, 
                                                'Fe',
                                                supercell=5,
                                                sphereradius=2.5)

Importing structure
Finding site inside/outside sphere.
Writing capped cluster file:  Staurolite3_capped.xyz
Writing bq charge file:  Staurolite3_capped.bq


## Epidote

In [44]:
cappingdistances = {
    'Fe': 2.03,
    'O': 0.98,
    'Si': 1.49,
    'Al': 1.93,
    'Ca': 2.36,
    'Pb': 1.73
}

#checkdist tells the program that the bond involves atoms which do not have unique symmetry sties and that it should defer to bq4unique to determine the bqcharge to place.
bqchargemap = {
    'O':{'Fe': +3/6 - 1,'Si':+4/4 - 1 ,'Al':+3/6 - 1, 'Pb':+2/10 - 1, 'Ca':+2/10 - 1}, # +2 charge / 6 bonds - 1 to override capping H
    'Fe':{'O': -3/6 - 1}, # -2 charge / 6 bonds - 1 to override capping H
    'Si':{'O':-4/4 - 1},
    'Al':{'O':-3/6 - 1},
    'Pb':{'O':-2/10 - 1},
    'Ca':{'O':-2/9 - 1}
}

bq4unique = {'Pb':{1:0}, #note, this is the atom being replaced and whether the oxygen involved is one, two, or three-fold coordinated
    'Ca':{1:0},
}

numnearest = {
    'Fe': 6,
    'O': -2,
    'Si': 4,
    'Al': 6,
    'Pb':-1,
    'Ca':-1
}

#looks for atoms within r+/-dr of the interior atom to decide they are bonding 
bondlength = {'r1':2.65,'dr1':0.45,'r2':1.8,'dr2':0.46}

flagAtoms = ['Pb', 'Ca']

In [45]:
capped = make_hydrogen_capped_cluster_from_cif_4('Epidote_capped',
                                               'epidote.cif',
                                                numnearest,
                                                bondlength, 
                                                cappingdistances, 
                                                bqchargemap,
                                                bq4unique, 
                                                'Fe',
                                                flagAtoms,
                                                supercell=5,
                                                sphereradius=5.8)

Importing structure
Finding site inside/outside sphere.
Writing capped cluster file:  Epidote_capped.xyz
Writing bq charge file:  Epidote_capped.bq
